In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd

df = pd.read_excel('/content/drive/MyDrive/Credit Vision/Operations Foresics/Temporal_Null_Analysis_Result.xlsx')
display(df.head())

,issue_year,total_records_for_year,id_non_null_count,id_non_null_pct,member_id_non_null_count,member_id_non_null_pct,loan_amnt_non_null_count,loan_amnt_non_null_pct,funded_amnt_non_null_count,funded_amnt_non_null_pct,...,settlement_status_non_null_count,settlement_status_non_null_pct,settlement_date_non_null_count,settlement_date_non_null_pct,settlement_amount_non_null_count,settlement_amount_non_null_pct,settlement_percentage_non_null_count,settlement_percentage_non_null_pct,settlement_term_non_null_count,settlement_term_non_null_pct
0,2007,603,603,100,0,0,603,100,603,100,...,1,0.17,1,0.17,1,0.17,1,0.17,1,0.17
1,2008,2393,2393,100,0,0,2393,100,2393,100,...,5,0.21,5,0.21,5,0.21,5,0.21,5,0.21
2,2009,5281,5281,100,0,0,5281,100,5281,100,...,12,0.23,12,0.23,12,0.23,12,0.23,12,0.23
3,2010,12537,12537,100,0,0,12537,100,12537,100,...,47,0.37,47,0.37,47,0.37,47,0.37,47,0.37
4,2011,21721,21721,100,0,0,21721,100,21721,100,...,95,0.44,95,0.44,95,0.44,95,0.44,95,0.44


In [11]:
import pandas as pd
import numpy as np

def create_yoy_heatmap_pct(df: pd.DataFrame):
    """
    Takes the temporal nullity data and calculates the Year-over-Year percentage
    change in the NON-NULL PERCENTAGES, visualizing the result as a diverging heatmap.
    This method correctly isolates changes in data collection policy from general
    business growth.

    Args:
        df (pd.DataFrame): The DataFrame loaded from the SQL query results.

    Returns:
        A styled pandas DataFrame (Styler object) representing the heatmap.
    """
    print("Step 1: Preparing and cleaning the percentage data...")
    # Select only the year and the non-null PERCENTAGE columns
    pct_cols = ['issue_year'] + [col for col in df.columns if col.endswith('_non_null_pct')]
    pct_df = df[pct_cols].copy()

    # Clean up column names for better readability
    cleaned_columns = {col: col.replace('_non_null_pct', '') for col in pct_cols}
    pct_df.rename(columns=cleaned_columns, inplace=True)

    # Set the year as the index for time-series operations
    pct_df.set_index('issue_year', inplace=True)

    print("Step 2: Calculating YoY percentage change of the percentages...")
    # Use the built-in pct_change() and multiply by 100
    yoy_df = pct_df.pct_change() * 100

    print("Step 3: Cleaning the calculated data (handling NaN and Infinity)...")
    # Replace infinite values (from 0% to >0%) with a large number for emphasis
    # and NaN values (from the first year) with 0.
    yoy_df.replace([np.inf, -np.inf], 9999, inplace=True)
    yoy_df.fillna(0, inplace=True)

    print("Step 4: Generating the diverging heatmap...")
    # Transpose the DataFrame so features are rows and years are columns
    # Using a diverging colormap where red is a decrease in collection rate,
    # and blue is an increase.
    styled_heatmap = yoy_df.T.style.background_gradient(cmap='coolwarm', axis=None, vmin=-100, vmax=200)\
                                .format("{:,.1f}%")\
                                .set_caption("<b>YoY % Change in Data Point Collection Rate</b>")

    print("\n YoY Percentage Analysis Complete.")
    return styled_heatmap, yoy_df

In [13]:
# Generate the heatmap
yoy_heatmap, raw_yoy = create_yoy_heatmap_pct(df)

# Display the result
display(yoy_heatmap)

Step 1: Preparing and cleaning the percentage data...
Step 2: Calculating YoY percentage change of the percentages...
Step 3: Cleaning the calculated data (handling NaN and Infinity)...
Step 4: Generating the diverging heatmap...

 YoY Percentage Analysis Complete.


issue_year,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
id,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
member_id,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
loan_amnt,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
funded_amnt,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
funded_amnt_inv,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
term,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
int_rate,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
installment,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
grade,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
sub_grade,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%


In [12]:
import pandas as pd
import numpy as np
from collections import defaultdict
import json

def generate_operational_forensics_report(df_original, yoy_df, threshold=5.0, new_column_threshold=1000.0):
    """
    Generate comprehensive operational forensics report analyzing data collection evolution.

    Args:
        df_original (pd.DataFrame): Original DataFrame with count/percentage columns
        yoy_df (pd.DataFrame): YoY percentage change DataFrame (years=index, features=columns)
        threshold (float): Fluctuation threshold for significant changes
        new_column_threshold (float): Threshold to identify newly introduced columns

    Returns:
        dict: Complete forensics analysis report
    """

    # Initialize report structure
    report = {
        'metadata': {
            'analysis_period': f"{yoy_df.index.min()}-{yoy_df.index.max()}",
            'total_years': len(yoy_df.index),
            'total_features': len(yoy_df.columns),
            'fluctuation_threshold': threshold,
            'new_column_threshold': new_column_threshold
        },
        'yearly_analysis': {},
        'feature_lifecycle': {},
        'summary_statistics': {}
    }

    # Create feature mapping for original data lookup
    feature_mapping = {}
    for col in df_original.columns:
        if col.endswith('_non_null_pct'):
            clean_name = col.replace('_non_null_pct', '')
            if clean_name in yoy_df.columns:
                feature_mapping[clean_name] = {
                    'pct_col': col,
                    'count_col': col.replace('_non_null_pct', '_non_null_count')
                }

    # Set up original data lookup
    df_original_indexed = df_original.set_index('issue_year')

    # Years are in the index of yoy_df
    years = sorted(yoy_df.index)

    # Year-by-year analysis
    for i, year in enumerate(years):
        # Get year changes - this returns a Series with features as index
        year_changes = yoy_df.loc[year]

        # Get total records for this year
        total_records = None
        if 'total_records_for_year' in df_original_indexed.columns:
            total_records = int(df_original_indexed.loc[year, 'total_records_for_year'])

        year_data = {
            'year': int(year),
            'total_records': total_records,
            'new_columns': [],
            'significant_fluctuations': [],
            'deprecated_columns': []
        }

        # Identify new columns (very high percentage increases)
        new_cols = year_changes[year_changes >= new_column_threshold]

        # Identify significant fluctuations (excluding new columns)
        fluctuating_cols = year_changes[
            (abs(year_changes) >= threshold) &
            (year_changes < new_column_threshold)
        ]

        # Identify deprecated columns (large negative changes)
        deprecated_cols = year_changes[year_changes <= -50.0]

        # Process new columns
        for feature in new_cols.index:
            if feature in feature_mapping:
                pct_col = feature_mapping[feature]['pct_col']
                count_col = feature_mapping[feature]['count_col']

                current_pct = df_original_indexed.loc[year, pct_col] if pct_col in df_original_indexed.columns else 0
                current_count = df_original_indexed.loc[year, count_col] if count_col in df_original_indexed.columns else 0

                year_data['new_columns'].append({
                    'feature': feature,
                    'yoy_change_pct': float(new_cols[feature]),
                    'current_percentage': float(current_pct),
                    'current_count': int(current_count),
                    'previous_percentage': 0.0,
                    'previous_count': 0
                })

        # Process significant fluctuations
        for feature in fluctuating_cols.index:
            if feature in feature_mapping:
                pct_col = feature_mapping[feature]['pct_col']
                count_col = feature_mapping[feature]['count_col']

                current_pct = df_original_indexed.loc[year, pct_col] if pct_col in df_original_indexed.columns else 0
                current_count = df_original_indexed.loc[year, count_col] if count_col in df_original_indexed.columns else 0

                # Get previous year data
                prev_pct = 0
                prev_count = 0
                if i > 0:
                    prev_year = years[i-1]
                    prev_pct = df_original_indexed.loc[prev_year, pct_col] if pct_col in df_original_indexed.columns else 0
                    prev_count = df_original_indexed.loc[prev_year, count_col] if count_col in df_original_indexed.columns else 0

                year_data['significant_fluctuations'].append({
                    'feature': feature,
                    'yoy_change_pct': float(fluctuating_cols[feature]),
                    'current_percentage': float(current_pct),
                    'current_count': int(current_count),
                    'previous_percentage': float(prev_pct),
                    'previous_count': int(prev_count),
                    'absolute_change': float(current_pct - prev_pct),
                    'direction': 'increase' if fluctuating_cols[feature] > 0 else 'decrease'
                })

        # Process deprecated columns
        for feature in deprecated_cols.index:
            if feature in feature_mapping:
                pct_col = feature_mapping[feature]['pct_col']
                count_col = feature_mapping[feature]['count_col']

                current_pct = df_original_indexed.loc[year, pct_col] if pct_col in df_original_indexed.columns else 0
                current_count = df_original_indexed.loc[year, count_col] if count_col in df_original_indexed.columns else 0

                year_data['deprecated_columns'].append({
                    'feature': feature,
                    'yoy_change_pct': float(deprecated_cols[feature]),
                    'current_percentage': float(current_pct),
                    'current_count': int(current_count),
                    'severity': 'complete_removal' if current_pct < 1.0 else 'significant_decline'
                })

        # Year summary
        year_data['summary'] = {
            'new_features_count': len(year_data['new_columns']),
            'fluctuating_features_count': len(year_data['significant_fluctuations']),
            'deprecated_features_count': len(year_data['deprecated_columns'])
        }

        report['yearly_analysis'][int(year)] = year_data

    # Feature lifecycle analysis - features are columns in yoy_df
    feature_lifecycles = {}
    for feature in yoy_df.columns:
        # Get feature data across all years
        feature_data = yoy_df[feature]

        lifecycle = {
            'feature': feature,
            'introduction_year': None,
            'deprecation_year': None,
            'lifecycle_stage': 'stable',
            'volatility_score': float(feature_data.std()),
            'total_changes': int((abs(feature_data) >= threshold).sum()),
            'years_active': int((feature_data != 0).sum())
        }

        # Find introduction year (first significant increase)
        intro_candidates = feature_data[feature_data >= new_column_threshold]
        if not intro_candidates.empty:
            lifecycle['introduction_year'] = int(intro_candidates.index[0])
            lifecycle['lifecycle_stage'] = 'introduced'

        # Find deprecation year (large negative change)
        deprecation_candidates = feature_data[feature_data <= -50.0]
        if not deprecation_candidates.empty:
            lifecycle['deprecation_year'] = int(deprecation_candidates.index[-1])
            if lifecycle['lifecycle_stage'] != 'introduced':
                lifecycle['lifecycle_stage'] = 'deprecated'

        # Determine current stage
        if lifecycle['introduction_year'] and not lifecycle['deprecation_year']:
            lifecycle['lifecycle_stage'] = 'active'
        elif lifecycle['introduction_year'] and lifecycle['deprecation_year']:
            lifecycle['lifecycle_stage'] = 'deprecated'

        feature_lifecycles[feature] = lifecycle

    report['feature_lifecycle'] = feature_lifecycles

    # Summary statistics
    all_new_features = []
    all_fluctuations = []
    expansion_years = 0

    for year_data in report['yearly_analysis'].values():
        all_new_features.extend(year_data['new_columns'])
        all_fluctuations.extend(year_data['significant_fluctuations'])
        if year_data['summary']['new_features_count'] > 0:
            expansion_years += 1

    # Find most active years
    most_expansive_year = max(report['yearly_analysis'].keys(),
                             key=lambda y: report['yearly_analysis'][y]['summary']['new_features_count'])
    most_volatile_year = max(report['yearly_analysis'].keys(),
                            key=lambda y: report['yearly_analysis'][y]['summary']['fluctuating_features_count'])

    report['summary_statistics'] = {
        'total_new_features_introduced': len(all_new_features),
        'total_significant_fluctuations': len(all_fluctuations),
        'years_with_platform_expansion': expansion_years,
        'most_expansive_year': most_expansive_year,
        'most_volatile_year': most_volatile_year,
        'platform_evolution_score': len(all_new_features) / len(years) if len(years) > 0 else 0
    }

    return report

def run_forensics_analysis(df_original, yoy_df, threshold=5.0):
    """
    Execute complete forensics analysis workflow
    """
    print("Generating Operational Forensics Report...")

    forensics_report = generate_operational_forensics_report(
        df_original=df_original,
        yoy_df=yoy_df,
        threshold=threshold,
        new_column_threshold=1000.0
    )

    print("Analysis Complete.")
    return forensics_report

def print_report_summary(report):
    """
    Print formatted summary of forensics findings
    """
    print("\nOPERATIONAL FORENSICS REPORT SUMMARY")
    print("=" * 60)

    print(f"\nAnalysis Period: {report['metadata']['analysis_period']}")
    print(f"Total Features: {report['metadata']['total_features']}")
    print(f"Total Years: {report['metadata']['total_years']}")

    print(f"\nKEY FINDINGS:")
    print(f"New Features Introduced: {report['summary_statistics']['total_new_features_introduced']}")
    print(f"Most Expansive Year: {report['summary_statistics']['most_expansive_year']}")
    print(f"Most Volatile Year: {report['summary_statistics']['most_volatile_year']}")
    print(f"Platform Evolution Score: {report['summary_statistics']['platform_evolution_score']:.2f}")

    print(f"\nYEAR-BY-YEAR BREAKDOWN:")
    for year, data in sorted(report['yearly_analysis'].items()):
        print(f"{year}: {data['summary']['new_features_count']} new, "
              f"{data['summary']['fluctuating_features_count']} fluctuating, "
              f"{data['summary']['deprecated_features_count']} deprecated")

def export_report_data(report, yoy_df):
    """
    Export report data to files
    """
    # Export complete report
    with open('forensics_report.json', 'w') as f:
        json.dump(report, f, indent=2)

    # Export YoY data
    yoy_df.to_csv('yoy_changes.csv')

    # Export yearly summary
    summary_data = []
    for year, data in report['yearly_analysis'].items():
        summary_data.append({
            'Year': year,
            'Total_Records': data.get('total_records', None),
            'New_Features': data['summary']['new_features_count'],
            'Fluctuating_Features': data['summary']['fluctuating_features_count'],
            'Deprecated_Features': data['summary']['deprecated_features_count']
        })

    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv('yearly_summary.csv', index=False)

    print("Data exported to: forensics_report.json, yoy_changes.csv, yearly_summary.csv")

# Execution wrapper
def execute_complete_analysis(df):
    """
    Execute complete forensics analysis pipeline
    """
    print("Starting Complete Forensics Analysis")
    print("-" * 50)

    # Step 1: Generate YoY heatmap
    print("Step 1: Generating YoY Heatmap")
    yoy_heatmap, yoy_df = create_yoy_heatmap_pct(df)

    # Step 2: Generate forensics report
    print("\nStep 2: Generating Forensics Report")
    forensics_report = run_forensics_analysis(df, yoy_df)

    # Step 3: Print summary
    print_report_summary(forensics_report)

    # Step 4: Export data
    print("\nStep 3: Exporting Data")
    export_report_data(forensics_report, yoy_df)

    return yoy_heatmap, yoy_df, forensics_report

In [14]:
yoy_heatmap, yoy_df, forensics_report = execute_complete_analysis(df)

Starting Complete Forensics Analysis
--------------------------------------------------
Step 1: Generating YoY Heatmap
Step 1: Preparing and cleaning the percentage data...
Step 2: Calculating YoY percentage change of the percentages...
Step 3: Cleaning the calculated data (handling NaN and Infinity)...
Step 4: Generating the diverging heatmap...

 YoY Percentage Analysis Complete.

Step 2: Generating Forensics Report
Generating Operational Forensics Report...
Analysis Complete.

OPERATIONAL FORENSICS REPORT SUMMARY

Analysis Period: 2007-2018
Total Features: 150
Total Years: 12

KEY FINDINGS:
New Features Introduced: 101
Most Expansive Year: 2012
Most Volatile Year: 2013
Platform Evolution Score: 8.42

YEAR-BY-YEAR BREAKDOWN:
2007: 0 new, 0 fluctuating, 0 deprecated
2008: 1 new, 22 fluctuating, 1 deprecated
2009: 0 new, 10 fluctuating, 2 deprecated
2010: 0 new, 10 fluctuating, 0 deprecated
2011: 0 new, 10 fluctuating, 1 deprecated
2012: 51 new, 8 fluctuating, 0 deprecated
2013: 1 new,